In [26]:

import os
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

importing all the requied packages

read the data set into articles,customers,and transaction variable

In [27]:
%%time
articles_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
customers_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")
sample_submission_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv")

transactions_train_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

articel data set coloumns

In [28]:
articles_df.head()

cusomers dataset columns

In [29]:
customers_df.head()

In [30]:
transactions_train_df.head()

In [31]:
sample_submission_df.head(3).style.set_properties(**{'background-color': 'rgba(184,230,194,.5)'})

Transactions table is the train data. It contains customer_id and article_id, which are foreign keys for the customer and articles tables. Also, Transactions also contains sales_channel_id.

Here we can check missing data, count unique values etc.

In [32]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

def unique_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    uniques = []
    for col in data.columns:
        unique = data[col].nunique()
        uniques.append(unique)
    tt['Uniques'] = uniques
    return tt

 we can look through missing data in different files.

In the articles_df, the only missing data is for the detailed description of the article (0.4% missing data).

In the cutomers_df, only customer_id and postal_code are completely filled. Age, fashion_news_frequency have around 1% misssing data, FN has 65% missing and Active has 66% missing data.

In the transactions_train_df, there is no missing data.

In [33]:
missing_data(articles_df).head(25)

In [34]:
missing_data(customers_df).style.set_properties(**{'background-color': 'rgba(245, 245, 152,.5)'})

In [35]:
missing_data(transactions_train_df).style.set_properties(**{'background-color': 'rgba(152, 243, 245,.5)'})

After that we can check unique values:

In [36]:

unique_values(articles_df).head(5).style.set_properties(**{'background-color': 'rgba(145, 178, 227,.5)'})

In [37]:
unique_values(customers_df).head(5).style.set_properties(**{'background-color': 'rgba(130, 126, 230,.5)'})

In [38]:
unique_values(transactions_train_df).head(5).style.set_properties(**{'background-color': 'rgba(188, 126, 230,.5)'})

**observation:**
We can see that not all the customers in customer data are appearing as having transactions in transaction train data. Moreover, not all articles are represented in this data.

It is interesting that the number of different prices is quite small, out of 31.7M transactions, and for 1.3M customers, buying 104K different articles. Same for the dates, there are only 734 different dates.

DATA VISUALIZATION FOR CUSTOMER DATAFRAME

# PIECHART AND BAR_PLOT FUNCTION FOR DATA FRAME
# 

In [39]:
def pie_chart(df, col_values, labels, ax, color, title):
    n_classes = len(df)
    explode = (0.1,) * n_classes # explode for 0.1 each slice
    ax.pie(df[col_values],
           colors=color, 
           explode=explode,
           labels=df[labels],
           shadow=True)
    ax.set_title(title, fontsize=16)
    
def bar_plot(df, col_x, col_y, ax, color, title):
    ax.bar(x=df[col_x],
           height=df[col_y],
           color=color)
    ax.set_title(title, fontsize=16) 
    plt.xticks(rotation=90)

# # PICHART FOR ARTICLE

In [40]:
temp = articles_df.groupby(["product_group_name"])["product_type_name"].nunique()
df = pd.DataFrame({'Product Group': temp.index,'Product Types': temp.values})
df = df.sort_values(['Product Types'], ascending=False)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20,10))
color = plt.cm.autumn(np.linspace(0, 1, len(df)))

pie_chart(df,
          'Product Types', 
          'Product Group',
          axes, 
          color,  
          "Product Types per each Product Group")  

In [41]:
temp = articles_df.groupby(["product_group_name"])["article_id"].nunique()
df = pd.DataFrame({'Product Group': temp.index,'Articles': temp.values})
df = df.sort_values(['Articles'], ascending=False)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20,10))
color = plt.cm.summer(np.linspace(0, 1, len(df)))

pie_chart(df,
          'Articles', 
          'Product Group',
          axes, 
          color,  
          "Product Types per each Product Group")  

 # ARTICLE DATAFRAME  INDEX_GROUP_NAME &ARTICLE ID(UNIQUE VALUE)

In [42]:
temp = articles_df.groupby(["index_group_name"])["article_id"].nunique()
df = pd.DataFrame({'Index Group Name': temp.index,'Articles': temp.values})
df = df.sort_values(['Articles'], ascending=False)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
color = plt.cm.spring(np.linspace(0, 1, len(df)))

bar_plot(df,
         'Index Group Name',
         'Articles',
         axes, 
         color, 
         "Number of Articles per each Index Group Name")

HERE WE CAN OBSERVE THE 19 TO 26 AGE PEOPLE BUY MOST OF THE PRODUCT 
'




SECONDLY BETWEEN 45 TO 55 AGE GROUP PEOPLE BUY AVERAGE PRODUCT

In [43]:
temp = articles_df.groupby(["product_type_name"])["article_id"].nunique()
df = pd.DataFrame({'Product Type': temp.index,'Articles': temp.values})
total_types = len(df['Product Type'].unique())
df = df.sort_values(['Articles'], ascending=False)[0:30]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,6))
color = plt.cm.cool(np.linspace(0, 1, len(df)))

bar_plot(df,
         'Product Type',
         'Articles',
         axes, 
         color, 
         "Number of Articles per each Product Type (top 30)")

In [44]:
temp = articles_df.groupby(["department_name"])["article_id"].nunique()
df = pd.DataFrame({'Department Name': temp.index,'Articles': temp.values})
total_depts = len(df['Department Name'].unique())
df = df.sort_values(['Articles'], ascending=False).head(20)


fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(22,10))
color = plt.cm.pink(np.linspace(0, 1, len(df)))

pie_chart(df,
          'Articles', 
          'Department Name',
          axes, 
          color,  
          "Number of Articles per each Department (top 20)")  

In [45]:
temp = articles_df.groupby(["section_name"])["article_id"].nunique()
df = pd.DataFrame({'Section Name': temp.index,'Articles': temp.values})
df = df.sort_values(['Articles'], ascending=False)


fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(22,10))
color = plt.cm.PRGn(np.linspace(0, 1, len(df)))

pie_chart(df.head(15),
          'Articles', 
          'Section Name',
          axes, 
          color,  
          "Number of Articles per each Section Name (top 15)")  

In [46]:
temp = articles_df.groupby(["graphical_appearance_name"])["article_id"].nunique()
df = pd.DataFrame({'Graphical Appearance Name': temp.index,'Articles': temp.values})
df = df.sort_values(['Articles'], ascending=False).head(50)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(22,10))
color = plt.cm.PiYG(np.linspace(0, 1, len(df)))

pie_chart(df.head(15),
          'Articles', 
          'Graphical Appearance Name',
          axes, 
          color,  
          "Number of Articles per each Graphical Appearance Name (top 15)")  

# CUSTOMER TABLE DATA VISUALIZATION
# 

In [47]:
temp = customers_df.groupby(["age"])["customer_id"].count()
df = pd.DataFrame({'Age': temp.index,'Customers': temp.values})
df = df.sort_values(['Age'], ascending=False)


fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,6))
color = plt.cm.cool(np.linspace(0, 1, len(df)))

bar_plot(df,
         'Age',
         'Customers',
         axes, 
         color, 
         "Number of Customers per each Age")

In [48]:

temp = customers_df.groupby(["fashion_news_frequency"])["customer_id"].count()
df = pd.DataFrame({'Fashion News Frequency': temp.index,'Customers': temp.values})
df = df.sort_values(['Customers'], ascending=False)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(4,6))
color = plt.cm.seismic(np.linspace(0, 1, len(df)))

bar_plot(df,
         'Fashion News Frequency',
         'Customers',
         axes, 
         color, 
         "Number of Customers per each Fashion News Frequency")

# Data visualisations. Transactions Data 📊

In [49]:
df = transactions_train_df.sample(100_000)
fig, ax = plt.subplots(1, 1, figsize=(14, 7))
sns.kdeplot(np.log(df.loc[df["sales_channel_id"]==1].price.value_counts()),
           color="red")
sns.kdeplot(np.log(df.loc[df["sales_channel_id"]==2].price.value_counts()),
           color="blue")

ax.legend(labels=['Sales channel 1', 
                  'Sales channel 2'])

plt.title("Logaritmic distribution of price frequency \
in transactions, grouped per sales channel (100k sample)")

plt.show()

In [50]:
df = transactions_train_df.sample(100_000).groupby(["t_dat"])["article_id"].count().reset_index()
df["t_dat"] = df["t_dat"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df.columns = ["Date", "Transactions"]

fig, ax = plt.subplots(1, 1, figsize=(16,6))
plt.plot(df["Date"], df["Transactions"], color="red")
plt.xlabel("Date")
plt.ylabel("Transactions")
plt.title(f"Transactions per day (100k sample)")
plt.show()

In [51]:
df = transactions_train_df.sample(100_000).groupby(["t_dat", "sales_channel_id"])["article_id"].count().reset_index()
df["t_dat"] = df["t_dat"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

df.columns = ["Date", "Sales Channel Id", "Transactions"]

fig, ax = plt.subplots(1, 1, figsize=(16,6))
g1 = ax.plot(df.loc[df["Sales Channel Id"]==1, "Date"], 
             df.loc[df["Sales Channel Id"]==1, 
                    "Transactions"], 
             label="Sales Channel 1", 
             color="Blue")

g2 = ax.plot(df.loc[df["Sales Channel Id"]==2, "Date"], 
             df.loc[df["Sales Channel Id"]==2, 
                    "Transactions"], 
             label="Sales Channel 2", 
             color="Red")

plt.xlabel("Date")
plt.ylabel("Transactions")
ax.legend()
plt.title(f"Transactions per day, grouped by Sales Channel (100k sample)")
plt.show()

In [52]:
df = transactions_train_df.groupby(["t_dat", "sales_channel_id"])["article_id"].nunique().reset_index()
df["t_dat"] = df["t_dat"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df.columns = ["Date", "Sales Channel Id", "Unique Articles"]

fig, ax = plt.subplots(1, 1, figsize=(16,6))
g1 = ax.plot(df.loc[df["Sales Channel Id"]==1, 
                    "Date"], 
             df.loc[df["Sales Channel Id"]==1, 
                    "Unique Articles"], 
             label="Sales Channel 1", 
             color="Blue")

g2 = ax.plot(df.loc[df["Sales Channel Id"]==2, 
                    "Date"], 
             df.loc[df["Sales Channel Id"]==2, 
                    "Unique Articles"], 
             label="Sales Channel 2", 
             color="Orange")

plt.xlabel("Date")
plt.ylabel("Unique Articles / Day")
ax.legend()
plt.title(f"Unique articles per day, grouped by Sales Channel")
plt.show()

# IMAGE DATA ANALYSIS 


In [53]:
total_folders = 0
total_files = 0

folder_info = []
images_names = []

path = "../input/h-and-m-personalized-fashion-recommendations"

for base, dirs, files in tqdm(os.walk(path)):
    for directories in dirs:
        folder_info.append((directories, 
                            len(os.listdir(os.path.join(base, directories)))))
        total_folders = total_folders + 1
    
    for _files in files:
        total_files = total_files + 1
        if (len(_files.split(".jpg"))==2):
            images_names.append(_files.split(".jpg")[0])

In [54]:
print(f"• Total number of folders: {total_folders}")
print(f"• Total number of files: {total_files}")

In [55]:
folder_info_df = pd.DataFrame(folder_info, 
                              columns=["folder", 
                                       "files count"])

folder_info_df.sort_values(["files count"], ascending=False).head().style.set_properties(**{'background-color': 'rgba(184,230,194,.5)'})

In [56]:
image_name_df = pd.DataFrame(images_names, columns = ["image_name"])
image_name_df["article_id"] = image_name_df["image_name"].apply(lambda x: int(x[1:]))
image_name_df.head().style.set_properties(**{'background-color': 'rgba(184,230,194,.5)'})

In [57]:
image_article_df = articles_df[["article_id", 
                                "product_code", 
                                "product_group_name", 
                                "product_type_name"]].merge(image_name_df, 
                                                            on=["article_id"], 
                                                            how="left")
image_article_df.head(50).style.set_properties(**{'background-color': 'rgba(184,230,194,.5)'})

In [58]:
image_article_df.nunique()

In [59]:
# image_article_df.product_group_name .nunique()

In [60]:
def plot_image_samples(image_article_df, product_group_name, cols=1, rows=-1):
    image_path = "../input/h-and-m-personalized-fashion-recommendations/images/"
    _df = image_article_df.loc[image_article_df.product_group_name==product_group_name]
    article_ids = _df.article_id.values[0:cols*rows]
    plt.figure(figsize=(2 + 3 * cols, 2 + 4 * rows))
    for i in range(cols * rows):
        article_id = ("0" + str(article_ids[i]))[-10:]
        plt.subplot(rows, cols, i + 1)
        plt.axis('off')
        plt.title(f"{product_group_name} {article_id[:3]}\n{article_id}.jpg")
        image = Image.open(f"{image_path}{article_id[:3]}/{article_id}.jpg")
        plt.imshow(image)
        
plot_image_samples(image_article_df, "Garment Lower body", 5, 1)
plot_image_samples(image_article_df, "Accessories", 5, 1)
plot_image_samples(image_article_df, "Swimwear", 5, 1)
plot_image_samples(image_article_df, "Bags", 5, 1)

# RANDOM FOREST  
(DATA PREPARATION SAMPLE DATA 10K FROM TRANSACTION DATSET)

articles_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
customers_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")
sample_submission_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv")

transactions_train_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

load rthe data set with another variable name

In [70]:
articles = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
customers = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")
transactions = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

In [72]:
transactions['t_dat'].min()

In [ ]:

transactions['t_dat'].max()

In [ ]:
# 10000 samples of tansactions
df_sample = transactions.sample(n=10000)
df_sample.shape

In [ ]:

transactions.sort_values(by='price', ascending=False)['price'][:50]


In [ ]:

df_sample.head()

In [ ]:

# one month sample of data
# df_sample = transactions[transactions['t_dat'] > '2020-09-10']

# df_sample.shape
#join customer data

df_sample = pd.merge(df_sample, customers, on='customer_id')

In [ ]:
#join article data
# df_sample =pd.merge(df_sample, articles, on='article_id')
df_sample.head()


In [ ]:

# Count Na in %
df_sample.isnull().sum()/df_sample.isnull().count()*100

In [ ]:
print(df_sample['FN'].unique())
print(df_sample['Active'].unique())

In [ ]:

# fill nan in FN and Active columns with 0
df_sample['FN'] = df_sample['FN'].fillna(0)
df_sample['Active'] = df_sample['Active'].fillna(0)
df_sample.isnull().sum()/df_sample.isnull().count()*100

In [ ]:

#Imputate others with most frequen value 
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
imputed_df_sample = pd.DataFrame(imputer.fit_transform(df_sample))

imputed_df_sample.columns = df_sample.columns

print(imputed_df_sample.isnull().sum()/imputed_df_sample.isnull().count()*100)

In [ ]:
# Object data to category
for col in imputed_df_sample.select_dtypes(include=['object']).columns:
    imputed_df_sample[col] = imputed_df_sample[col].astype('category')

from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

object_cols = imputed_df_sample.select_dtypes(include=['category']).columns

# Encoding categorical features
# for col in imputed_df_sample.select_dtypes(include=['category']).columns:


imputed_df_sample[object_cols] = ordinal_encoder.fit_transform(imputed_df_sample[object_cols])
    
    
# t_dat to datetime objecct
imputed_df_sample['t_dat'] = pd.to_datetime(imputed_df_sample['t_dat'])
    
imputed_df_sample.info()

In [ ]:
imputed_df_sample

In [ ]:
# drop price to prevent data leakage
imputed_df_sample = imputed_df_sample.drop(['price'], axis=1)

In [ ]:
# plot correlation matrix 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

plt.figure(figsize=[7,5])
sns.heatmap(imputed_df_sample.corr())
plt.show()

# Random Forest

MODEL FIT XTRAIN AND XVALID

ref https://www.analyticsvidhya.com/blog/2021/06/understanding-random-forest/**

In [ ]:
from sklearn.model_selection import train_test_split

# select target and features
y = imputed_df_sample['article_id']
selected_columns = ['sales_channel_id', 'fashion_news_frequency' , 'club_member_status']

# spilt train and test data
X_train, X_valid, y_train, y_valid = train_test_split(imputed_df_sample[selected_columns], y, test_size = 0.3)

In [ ]:
X_train.head()

In [ ]:
X_train.shape, X_valid.shape

In [ ]:
y_train.shape, y_valid.shape

In [ ]:
#create model
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=150, n_jobs=1, max_depth=7, oob_score=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,
                                       n_estimators=100, oob_score=True)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
# Predicting
predict_labels = model.predict(X_valid)
print(predict_labels)

In [ ]:
X_valid.head()

In [ ]:
# checking the oob score
model.fit(X_train, y_train)

In [ ]:
model.oob_score_

evaluation mean absolute error of sample data set

In [ ]:
#evaluate
from sklearn.metrics import mean_absolute_error
mean_absolute_error(predict_labels, y_valid)

#  I dediced to follow logic, that was described in article

if there are articles for a certain client, pick the most recent buys;


if there are not articles for a certain client, just pick the most frequently buyed articles.

In [61]:
transactions_train_df = transactions_train_df.sort_values(["customer_id", 
                                                           "t_dat"], 
                                                          ascending=False)
transactions_train_df.head().style.set_properties(**{'background-color': 'rgba(184,230,194,.5)'})

> capture first what are the most frequent recently bought articles.

In [62]:
last_date = transactions_train_df.t_dat.max()
print(last_date)
print(transactions_train_df.loc[transactions_train_df.t_dat==last_date].shape)
print()

most_frequent_articles = list(transactions_train_df.loc[transactions_train_df.t_dat==last_date].article_id.value_counts()[0:12].index)
art_list = []
for art in most_frequent_articles:
    art = "0"+str(art)
    art_list.append(art)
art_str = " ".join(art_list)
print("Frequent articles bought recently:", art_str, end="\n")

In [63]:
agg_df = transactions_train_df.groupby(["customer_id"])["article_id"].agg(lambda x: str(x.values[0:12])[1:-1]).reset_index()

In [64]:
def padding_articles(x):
    if x:
        xl = x.split()
        x = []
        for xi in xl:
            x.append("0"+xi)
        dimm_x = len(x)
        if dimm_x < 12:
            x.extend(art_list[:12-dimm_x])
        return(" ".join(x))

In [65]:
agg_df["article_id"] = agg_df["article_id"].apply(lambda x: padding_articles(x))
print("Aggregated transaction history: ", agg_df.customer_id.nunique())
print("Submission sample: ", sample_submission_df.customer_id.nunique())

We'll replace the values in sample submission with the existent in aggregated transactions data and just let the default one otherwise.

sample submission csv file

In [66]:
sample_submission_df.head().style.set_properties(**{'background-color': 'rgba(184,230,194,.5)'})

In [67]:
submission_df = agg_df.merge(sample_submission_df[["customer_id"]], how="right")
submission_df.columns = ["customer_id", "prediction"]
print(submission_df.shape)
submission_df.head().style.set_properties(**{'background-color': 'rgba(184,230,194,.5)'})

In [68]:
print("Rows with missing data in submission: ", submission_df.loc[submission_df.prediction.isna()].shape[0])

replace the missing data with the most frequently bought articles, from recent days. We calculated it before

In [69]:
submission_df.loc[submission_df.prediction.isna(), ["prediction"]] = art_str
print("Rows with missing data in submission: ", submission_df.loc[submission_df.prediction.isna()].shape[0])
submission_df.to_csv("submission.csv", index=False)